In [ ]:
#for reg:linear    
#Optimization finished with maximum: 1.000000, at position: 
#{'colsample_bytree': 0.68465545509532988, 
#'learning_rate': 0.25209086974496175, 
#'max_delta_step': 0.029373671527204673, 
#'min_child_weight': 50.802215853240234, 
#'n_estimators': 465.11402637115225, 
#'subsample': 0.74770292804741545, 
#'max_depth': 11.40023258613906, 
#'gamma': 0.42648775254611759}.
#Time taken: 264 minutes and 57.92769 seconds.
#-----------------------------------------------------
#Final Results
#XGBOOST: 1.000000


#for count:poisson
#Optimization finished with maximum: 1.000000, at position: 
#{'colsample_bytree': 0.53776416509418445, 
#'learning_rate': 0.21307394122771003, 
#'max_delta_step': 0.077672683588664151, 
#'min_child_weight': 50.301184295946669, 
#'n_estimators': 455.68440847601119, 
#'subsample': 0.75168419234566375, 
#'max_depth': 6.0143000645742974, 
#'gamma': 0.13068999005755255}.
#Time taken: 160 minutes and 1.183995 seconds.
#-----------------------------------------------------
#Final Results
#XGBOOST: 1.000000

In [ ]:
#try model with/without PCA on continuous, etc after finding "best params" to see how the model changes or not
#if good change then rerun bayes_opt with PCA variables see how model may change max
#if not, submit and then try another ensemble approach with best of other models (RF, Extra Trees, Linear Reg, etc)
#and blend again with XGB or AdaBoost or whatever and see how that does

#if it all fails, get linux on backup drive and try autosklearn

#recommended next steps -- evaluating genetic algos, NNs.. personal strive is for development of abstracted general
#automatic ML algos (like auto-sklearn)



In [24]:
import time
t0 = time.clock()

import pandas as pd 
import numpy as np 
import xgboost as xgb
from scipy.optimize import fmin_powell
from ml_metrics import quadratic_weighted_kappa

def eval_wrapper(yhat, y):  
    y = np.array(y)
    y = y.astype(int)
    yhat = np.array(yhat)
    yhat = np.clip(np.round(yhat), np.min(y), np.max(y)).astype(int)   
    return quadratic_weighted_kappa(yhat, y)
    
def get_params():
    
    params = {}
    params["objective"] = "reg:linear"     
    params["eta"] = 0.038968208713327845
    params["gamma"] = 0.83745926973096763
    params["max_delta_step"] = 6.6502581223657851
    params["min_child_weight"] = 50.31287747915195
    params["subsample"] = 0.89026862324253775
    params["colsample_bytree"] = 0.86459387506389063
    params["silent"] = 1
    params["max_depth"] = 10.208631729376426
    params['n_estimators'] = 132.29526469869677
    plst = list(params.items())
    return plst

    return plst
    
def apply_offset(data, bin_offset, sv, scorer=eval_wrapper):
    # data has the format of pred=0, offset_pred=1, labels=2 in the first dim
    data[1, data[0].astype(int)==sv] = data[0, data[0].astype(int)==sv] + bin_offset
    score = scorer(data[1], data[2])
    return score

# global variables
columns_to_drop = ['Id', 'Response']
xgb_num_rounds = 500 #5000 gives me good score (~15 min), 10000 (~32 min) (no improvement)
num_classes = 8

print("Load the data using pandas")
DATA_DIR = '/Users/patrickkennedy/Desktop'

train = pd.read_csv(DATA_DIR + '/Project DATA/train.csv')
test = pd.read_csv(DATA_DIR + '/Project DATA/test.csv')

# combine train and test
all_data = train.append(test)

print('Eliminate missing values')    
# Use -1 for any others
all_data.fillna(-1, inplace=True)



# factorize categorical variables
# add all categorical vars -pk
# does this matter? and what of any feature selection beyond factorizing prod info 2?
categorical = ['Product_Info_1', 'Product_Info_2', 'Product_Info_3', 'Product_Info_5', 'Product_Info_6', 'Product_Info_7', 'Employment_Info_2', 'Employment_Info_3', 'Employment_Info_5', 'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 'InsuredInfo_6', 'InsuredInfo_7', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Family_Hist_1', 'Medical_History_2', 'Medical_History_3', 'Medical_History_4', 'Medical_History_5', 'Medical_History_6', 'Medical_History_7', 'Medical_History_8', 'Medical_History_9', 'Medical_History_10', 'Medical_History_11', 'Medical_History_12', 'Medical_History_13', 'Medical_History_14', 'Medical_History_16', 'Medical_History_17', 'Medical_History_18', 'Medical_History_19', 'Medical_History_20', 'Medical_History_21', 'Medical_History_22', 'Medical_History_23', 'Medical_History_25', 'Medical_History_26', 'Medical_History_27', 'Medical_History_28', 'Medical_History_29', 'Medical_History_30', 'Medical_History_31', 'Medical_History_33', 'Medical_History_34', 'Medical_History_35', 'Medical_History_36', 'Medical_History_37', 'Medical_History_38', 'Medical_History_39', 'Medical_History_40', 'Medical_History_41','Medical_History_1', 'Medical_History_15', 'Medical_History_24', 'Medical_History_32']



#does it matter that i am running on normalized variables? seems like it would obviate the need to do PCA...
#doing PCA on continuous variables
continuous = ['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI', 'Employment_Info_1', 'Employment_Info_4', 'Employment_Info_6',
              'Insurance_History_5', 'Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4', 'Family_Hist_5'] 


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X_scaled = StandardScaler().fit_transform(all_data[continuous])
pca = PCA(n_components=len(continuous), whiten=True).fit(X_scaled)
all_data_PCA = pca.transform(X_scaled)




PCA_titles = ['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11', 'PCA12', 'PCA13']
all_data_PCA_df = pd.DataFrame(all_data_PCA, index=all_data.index, columns=PCA_titles)
    
#add back into all_data and drop the continuous variables
all_data_new = pd.concat([all_data, all_data_PCA_df], axis=1)
all_data_new.drop(continuous, axis=1)



all_data_new['Product_Info_2'] = pd.factorize(all_data_new['Product_Info_2'])[0]

# fix the dtype on the label column
all_data_new['Response'] = all_data_new['Response'].astype(int)

# Provide split column
#all_data_new['Split'] = np.random.randint(5, size=all_data_new.shape[0])

# split train and test
train = all_data_new[all_data_new['Response']>0].copy()
test = all_data_new[all_data_new['Response']<1].copy()

    
# convert data to xgb data structure
xgtrain = xgb.DMatrix(train.drop(columns_to_drop, axis=1), train['Response'].values)
xgtest = xgb.DMatrix(test.drop(columns_to_drop, axis=1), label=test['Response'].values)    

# get the parameters for xgboost
plst = get_params()
print(plst)      

# train model
model = xgb.train(plst, xgtrain, xgb_num_rounds) 
test_preds = model.predict(xgtest, ntree_limit=model.best_iteration)


# get preds
train_preds = model.predict(xgtrain, ntree_limit=model.best_iteration)
print('Train score is:', eval_wrapper(train_preds, train['Response'])) 
test_preds = model.predict(xgtest, ntree_limit=model.best_iteration)
train_preds = np.clip(train_preds, -0.99, 8.99)
test_preds = np.clip(test_preds, -0.99, 8.99)

# train offsets 
offsets = np.ones(num_classes) * -0.5
offset_train_preds = np.vstack((train_preds, train_preds, train['Response'].values))

for j in range(num_classes):
    train_offset = lambda x: -apply_offset(offset_train_preds, x, j)
    offsets[j] = fmin_powell(train_offset, offsets[j])  


# apply offsets to test
data = np.vstack((test_preds, test_preds, test['Response'].values))
for j in range(num_classes):
    data[1, data[0].astype(int)==j] = data[0, data[0].astype(int)==j] + offsets[j]    
final_test_preds = np.round(np.clip(data[1], 1, 8)).astype(int)



preds_out = pd.DataFrame({"Id": test['Id'].values, "Response": final_test_preds})
preds_out = preds_out.set_index('Id')
preds_out.to_csv('xgb_offset_submission.csv')

print time.clock()-t0

Load the data using pandas
Eliminate missing values
[('colsample_bytree', 0.8645938750638906), ('silent', 1), ('max_delta_step', 6.650258122365785), ('min_child_weight', 50.31287747915195), ('n_estimators', 132.29526469869677), ('subsample', 0.8902686232425377), ('eta', 0.038968208713327845), ('objective', 'reg:linear'), ('max_depth', 10.208631729376426), ('gamma', 0.8374592697309676)]
('Train score is:', 0.7624300143942115)
Optimization terminated successfully.
         Current function value: -0.762430
         Iterations: 1
         Function evaluations: 14
Optimization terminated successfully.
         Current function value: -0.767026
         Iterations: 1
         Function evaluations: 14
Optimization terminated successfully.
         Current function value: -0.782899
         Iterations: 2
         Function evaluations: 40
Optimization terminated successfully.
         Current function value: -0.795946
         Iterations: 2
         Function evaluations: 56
Optimization termin

In [18]:
#clf = XGBRegressor seems to work a bit better

clf = xgb.XGBRegressor(learning_rate=0.01, min_child_weight=50, subsample=0.5, colsample_bytree=0.9, silent=True, max_depth=9, n_estimators=500)
xgb_model = clf.fit(train.drop(columns_to_drop, axis=1), train['Response'].values)
train_preds = xgb_model.predict(train.drop(columns_to_drop, axis=1))
test_preds = xgb_model.predict(test.drop(columns_to_drop, axis=1))

train_preds = np.clip(train_preds, -0.99, 8.99)
test_preds = np.clip(test_preds, -0.99, 8.99)

# train offsets 
offsets = np.ones(num_classes) * -0.5
offset_train_preds = np.vstack((train_preds, train_preds, train['Response'].values))

for j in range(num_classes):
    train_offset = lambda x: -apply_offset(offset_train_preds, x, j)
    offsets[j] = fmin_powell(train_offset, offsets[j])  


# apply offsets to test
data = np.vstack((test_preds, test_preds, test['Response'].values))
for j in range(num_classes):
    data[1, data[0].astype(int)==j] = data[0, data[0].astype(int)==j] + offsets[j]    
final_test_preds = np.round(np.clip(data[1], 1, 8)).astype(int)



preds_out = pd.DataFrame({"Id": test['Id'].values, "Response": final_test_preds})
preds_out = preds_out.set_index('Id')
preds_out.to_csv('xgb_offset_submission.csv')


Optimization terminated successfully.
         Current function value: -0.631287
         Iterations: 1
         Function evaluations: 14
Optimization terminated successfully.
         Current function value: -0.635976
         Iterations: 1
         Function evaluations: 14
Optimization terminated successfully.
         Current function value: -0.657183
         Iterations: 2
         Function evaluations: 40
Optimization terminated successfully.
         Current function value: -0.674084
         Iterations: 2
         Function evaluations: 55
Optimization terminated successfully.
         Current function value: -0.678185
         Iterations: 2
         Function evaluations: 53
Optimization terminated successfully.
         Current function value: -0.678235
         Iterations: 2
         Function evaluations: 51
Optimization terminated successfully.
         Current function value: -0.683800
         Iterations: 2
         Function evaluations: 47
Optimization terminated successful

In [19]:
test

,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,Family_Hist_1,Family_Hist_2,Family_Hist_3,...,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,PCA11,PCA12,PCA13
0,0.472262,0.150000,3,1,0.0000,2,0.5000,3,-1.000000,0.627451,...,1.660504,0.622386,1.673699,0.322613,-0.455705,0.925625,-1.300179,0.274549,0.149871,0.142802
1,0.484984,0.000000,1,3,0.0700,2,0.2000,2,-1.000000,0.529412,...,2.158894,-0.486344,-0.506636,0.564312,-1.010003,-0.119807,-0.594575,0.119875,0.265262,0.208149
2,0.519103,0.143000,9,1,0.0000,2,0.4500,3,0.666667,-1.000000,...,0.011103,-0.234593,0.185045,-0.848875,-1.089637,2.376914,-1.349909,0.077922,0.057385,0.242159
3,0.486962,0.210000,9,1,0.0000,2,1.0000,2,-1.000000,0.686275,...,2.087535,-0.477771,-0.159946,-1.116400,-0.221294,1.233872,1.283134,0.629774,-0.198439,0.215753
4,0.428718,0.085000,9,1,0.0000,2,0.2000,2,0.449275,-1.000000,...,-0.173927,0.025115,0.283592,-0.383632,-0.804490,1.513213,0.416392,0.149337,-0.143187,0.093398
5,0.379754,0.075000,9,1,0.0000,2,0.4000,3,-1.000000,0.647059,...,-0.740658,0.776569,1.545128,1.683839,-0.748370,0.601001,-0.092538,0.708732,0.062800,1.034871
6,0.373628,0.140000,9,1,0.0000,2,1.0000,2,0.449275,-1.000000,...,-0.249464,-0.810968,0.032149,-0.575579,-0.316176,-1.357708,-0.287404,-0.114073,0.098019,0.136405
7,0.388655,0.025000,9,1,0.0000,2,0.0000,2,0.391304,-1.000000,...,-2.169579,0.205734,0.791435,-0.032261,-0.181577,-0.277763,1.776499,0.768174,-0.072977,-0.185600
8,0.446361,0.035000,9,1,0.0000,2,-1.0000,3,-1.000000,0.656863,...,-0.435290,0.485178,0.980587,-0.109875,1.246287,0.489897,-1.049687,0.888814,0.036334,-0.224953
9,0.443418,0.060000,9,1,0.0000,2,1.0000,3,0.637681,-1.000000,...,-0.310795,0.127444,0.319312,0.417802,-2.444669,1.490517,0.544854,0.466325,-0.187974,0.404917


In [21]:
np.clip([1.2,2.7,3.3,4.3],2,3)

array([ 2. ,  2.7,  3. ,  3. ])

In [25]:
final_test_preds = np.round(np.clip(test_preds, 1, 8)).astype(int)

preds_out = pd.DataFrame({"Id": test['Id'].values, "Response": final_test_preds})
preds_out = preds_out.set_index('Id')
preds_out.to_csv('xgb_offset_submission.csv')

#BO gets me a score of .663... reliably, so that is good but not as good as i'd like
#offsets add about 0.01 to my final score so let's keep those in there.... next step is ensembling
#let's now choose the right model with hpsklearn and ultimately auto-sklearn

In [1]:
#once gridsearch and BO is done... let's do some evaluating with hyperopt and hpsklearn... this
#can tell us what preprocessing must be done, what classifier is best with what params too
#don't know how fast it is but at least it can give an idea of what model to include in the
#next ensemble

#also think about how i am using my cut points... np.digitize? how does this work again?